In [76]:
# 引入数据处理包
import numpy as np
import pandas as pd

# 机器学习
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, recall_score,precision_score

# 进行相关模型的保存
import joblib

In [77]:
# 表单3预测数据导入
data_toPre = pd.read_excel("附件.xlsx",sheet_name="表单3")

In [78]:
# 填补缺失数据
data_toPre.fillna(0,inplace=True)
data_toPre.head(2)

,文物编号,表面风化,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
0,A1,无风化,78.45,0.0,0.0,6.08,1.86,7.23,2.15,2.11,0.0,0.0,1.06,0.03,0.0,0.51
1,A2,风化,37.75,0.0,0.0,7.63,0.00,2.33,0.00,0.00,34.3,0.0,14.27,0.00,0.0,0.00


In [79]:
# 对数据进行85-105检查
sum_ele = data_toPre.iloc[:,2:16].sum(axis=1)
print(sum_ele)
print(data_toPre[(sum_ele < 85) & (sum_ele > 105)].shape[0])

0    99.48
1    96.28
2    98.98
3    96.00
4    99.62
5    99.10
6    99.64
7    99.98
dtype: float64
0


In [80]:
# 进行高钾与铅钡玻璃的划分
# 这里的判断并不是分类
# 是已经用spss分类后，这里简单的将其区分开
data_J_pre = data_toPre[data_toPre["氧化铅(PbO)"] <= 6].copy()
data_Q_pre = data_toPre[data_toPre["氧化铅(PbO)"] >= 6].copy()
data_Q_pre

,文物编号,表面风化,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
1,A2,风化,37.75,0.0,0.00,7.63,0.00,2.33,0.00,0.00,34.30,0.00,14.27,0.00,0.00,0.00
2,A3,无风化,31.95,0.0,1.36,7.19,0.81,2.93,7.06,0.21,39.58,4.69,2.68,0.52,0.00,0.00
3,A4,无风化,35.47,0.0,0.79,2.89,1.05,7.07,6.45,0.96,24.28,8.31,8.45,0.28,0.00,0.00
4,A5,风化,64.29,1.2,0.37,1.64,2.34,12.75,0.81,0.94,12.23,2.16,0.19,0.21,0.49,0.00
7,A8,无风化,51.12,0.0,0.23,0.89,0.00,2.12,0.00,9.01,21.24,11.34,1.46,0.31,0.00,2.26


In [81]:
part_J = ["文物编号","表面风化","二氧化硅(SiO2)","氧化钾(K2O)","氧化钙(CaO)","氧化铝(Al2O3)","氧化镁(MgO)","氧化铁(Fe2O3)","氧化铅(PbO)","五氧化二磷(P2O5)"]

part_Q = ["文物编号","表面风化","二氧化硅(SiO2)","氧化钠(Na2O)","氧化钙(CaO)","氧化铝(Al2O3)","氧化铅(PbO)","五氧化二磷(P2O5)"]

In [82]:
# 保存分类后的数据在excel表中
# 在excel结合问题一进行数据的风化前处理

excel_pre = pd.ExcelWriter("问题三预测数据_风化-未风化.xlsx")

data_J_pre_n = data_J_pre[part_J].copy()

data_Q_pre_n = data_Q_pre[part_Q].copy()

data_J_pre_n[data_J_pre_n["表面风化"]=="风化"].to_excel(excel_pre, "高钾_风化")

data_Q_pre_n[data_Q_pre_n["表面风化"]=="风化"].to_excel(excel_pre, "铅钡_风化")

excel_pre.save()
excel_pre.close()

In [83]:
# 引入未风化预测，整理后的数据

excel_get_aft = pd.ExcelFile("问题三预测数据处理后.xlsx")

data_J_rea = pd.read_excel(excel_get_aft, "高钾")
data_Q_rea = pd.read_excel(excel_get_aft, "铅钡")

excel_get_aft.close()

In [84]:
# 引入训练数据
data_J_toR = pd.read_excel("问题二聚类结果.xlsx",sheet_name="高钾",index_col=0)
data_Q_toR = pd.read_excel("问题二聚类结果.xlsx",sheet_name="铅钡",index_col=0)
data_J_col = pd.read_excel("问题二将亚分类数据.xlsx",sheet_name="高钾权重")
data_Q_col = pd.read_excel("问题二将亚分类数据.xlsx",sheet_name="铅钡权重")

In [85]:
J_need = np.append(data_J_col.columns.values,"k-means_label")
Q_need = np.append(data_Q_col.columns.values,"k-means_label")

In [86]:
# 选择出问题二中的分类特征进行分类
data_J_toR_af = data_J_toR[J_need].copy()
data_Q_toR_af = data_Q_toR[Q_need].copy()

In [87]:
# 用于进行给出附件3用于中结果
data_J_rea_to = data_J_rea[data_J_col.columns.values]
data_Q_rea_to = data_Q_rea[data_Q_col.columns.values]

In [88]:
# 进行高钾的多分类的训练与预测
x_train_J, x_test_J, y_train_J, y_test_J = train_test_split(data_J_toR_af.iloc[:,:-1],
                                                    data_J_toR_af.iloc[:,-1],
                                                    test_size=0.3,random_state=2)


xgb_J_ = XGBClassifier(
  num_class=3, learning_rate=0.1, max_depth=10, 
  objective='multi:softmax',min_child_weight=0,
  boosting='gbdt',alpha = 1,
)

xgb_J_.fit(x_train_J, y_train_J)

pre_y_test_J = xgb_J_.predict(x_test_J)
print(classification_report(y_test_J, pre_y_test_J))

[14:16:59] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "boosting" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         1

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



In [89]:
joblib.dump(xgb_J_, "J_xgboost.joblib")

['J_xgboost.joblib']

In [90]:
J_preRes = xgb_J_.predict(data_J_rea_to)

In [91]:
data_J_res = data_J_rea.copy()
print(J_preRes)
data_J_res["分类预测结果"] = J_preRes

[0 0 1]


In [65]:
# 进行bayes调参
# 贝叶斯调参
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

def xgb_cv(max_depth, learning_rate, n_estimators, min_child_weight, subsample, colsample_bytree, reg_alpha, gamma):
    val = cross_val_score(estimator=XGBClassifier(max_depth=int(max_depth),
                                                 learning_rate=learning_rate,
                                                 n_estimators=int(n_estimators),
                                                 min_child_weight=min_child_weight,
                                                #  subsample=max(min(subsample, 1), 0),
                                                #  colsample_bytree=max(min(colsample_bytree, 1), 0),
                                                 reg_alpha=max(reg_alpha, 0), gamma=gamma,
                                                 booster='gbtree',
                                                 objective='multi:softmax',
                                                 num_class=4,
                                                 seed=2022), X=data_Q_toR_af.iloc[:,:-1], y=data_Q_toR_af.iloc[:,-1], scoring="f1_macro",
                          cv=5).mean()
    return val

In [ ]:
xgb_bo = BayesianOptimization(xgb_cv, pbounds={'max_depth': (1, 10),
                                               'learning_rate': (0.01, 0.2),
                                               'n_estimators': (1, 100),
                                               'min_child_weight': (0, 15),
                                               'subsample': (0.001, 1),
                                               'colsample_bytree': (0.01, 1),
                                               'reg_alpha': (0.001, 10),
                                               'gamma': (0.001, 10)})
xgb_bo.maximize(n_iter=100, init_points=1000)

In [92]:
xgb_bo.max

{'target': 0.8261111111111111,
 'params': {'colsample_bytree': 0.10717903008046993,
  'gamma': 1.3191334532614911,
  'learning_rate': 0.17570002098068568,
  'max_depth': 9.365468069404896,
  'min_child_weight': 0.638013869379338,
  'n_estimators': 45.44870538575994,
  'reg_alpha': 0.601387103422647,
  'subsample': 0.7204545920421104}}

In [93]:
# xgboost_f1
res_xgb_best = cross_val_score(estimator=XGBClassifier(max_depth=9,
                                        learning_rate=0.17570002098068568,
                                        n_estimators=45,
                                        min_child_weight=0.638013869379338,
                                        reg_alpha=max(0.601387103422647, 0), gamma=1.3191334532614911,
                                        booster='gbtree',
                                        objective='multi:softmax',
                                        num_class=4,
                                        seed=2022), X=data_Q_toR_af.iloc[:,:-1], y=data_Q_toR_af.iloc[:,-1], scoring="f1_macro",
                          cv=5).mean()
res_xgb_best

0.8261111111111111

In [94]:
# 进行铅钡的多分类的训练与预测
x_train_Q, x_test_Q, y_train_Q, y_test_Q = train_test_split(data_Q_toR_af.iloc[:,:-1],
                                                    data_Q_toR_af.iloc[:,-1],
                                                    test_size=0.3,random_state=1)


xgb_Q_ = XGBClassifier(
  # num_class=4, learning_rate=0.1, max_depth=10, 
  # objective='multi:softmax',min_child_weight=0,
  # boosting='gbtree'
  max_depth=9,
  learning_rate=0.17570002098068568,
  n_estimators=45,
  min_child_weight=0.638013869379338,
  reg_alpha=max(0.601387103422647, 0), gamma=1.3191334532614911,
  booster='gbtree',
  objective='multi:softmax',
  num_class=4,
)

xgb_Q_.fit(x_train_Q, y_train_Q)

pre_y_test_Q  = xgb_Q_.predict(x_test_Q)
print(classification_report(y_test_Q, pre_y_test_Q))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       0.00      0.00      0.00         2
           2       0.67      1.00      0.80         4
           3       1.00      1.00      1.00         5

    accuracy                           0.83        12
   macro avg       0.67      0.75      0.70        12
weighted avg       0.72      0.83      0.77        12



D:\justwork\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\justwork\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\justwork\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [95]:
joblib.dump(xgb_Q_, "Q_xgboost.joblib")

['Q_xgboost.joblib']

In [223]:
Q_preRes = xgb_Q_.predict(data_Q_rea_to)
Q_preRes

array([3, 3, 3, 2, 3])

In [224]:
data_Q_res = data_Q_rea.copy()
print(Q_preRes)
data_Q_res["分类预测结果"] = Q_preRes

[3 3 3 2 3]


In [226]:
pre_Q3 = pd.ExcelWriter("问题三预测结果.xlsx")

data_J_res.to_excel(pre_Q3, "高钾")
data_Q_res.to_excel(pre_Q3, "铅钡")
pre_Q3.save()
pre_Q3.close()

In [75]:
# 与其他算法进行对比
#  与adaboost算法对比
from sklearn.ensemble import AdaBoostClassifier
#  与随机森林算法进行对比
from sklearn.ensemble import RandomForestClassifier
res_ada = cross_val_score(estimator=AdaBoostClassifier(random_state=2022), 
                X=data_Q_toR_af.iloc[:,:-1], y=data_Q_toR_af.iloc[:,-1], scoring="f1_macro",
                cv=5).mean()

print(res_ada)
res_rf = cross_val_score(estimator=RandomForestClassifier(random_state=2022), 
                X=data_Q_toR_af.iloc[:,:-1], y=data_Q_toR_af.iloc[:,-1], scoring="f1_macro",
                cv=5).mean()

print(res_rf)

0.5822619047619048
0.756547619047619
